## Populating decoding-related hex maze tables

So you've done decoding for your hex maze session and have a results xarray.
It's time to assign the decoded location to a hex so we can use it for analysis!

In [44]:
import datajoint as dj
import numpy as np

import spyglass.common as sgc
from spyglass.common import Nwbfile
from spyglass.utils.nwb_helper_fn import get_nwb_file

from hex_maze_decoding import DecodedPosition, DecodedHexPositionSelection, DecodedHexPosition

# Make sure the session exists
nwb_file_name = "IM-1478_20220726_.nwb"

# Fetch file create date and source version to make sure it's ok
nwb_file_abspath = Nwbfile.get_abs_path(nwb_file_name)
nwbf = get_nwb_file(nwb_file_abspath)
print(f"File created on {nwbf.file_create_date[0].strftime('%d/%m/%Y %H:%M:%S')}")
print(f"Source script version {nwbf.source_script}")

File created on 29/10/2025 18:00:06
Source script version jdb_to_nwb 2.1.2.dev2+g12a9b6eed.d20251030


View existing entries in hex maze decode related tables

In [45]:
display(DecodedPosition())

display(DecodedHexPositionSelection())

display(DecodedHexPosition())

decoding_merge_id,nwb_file_name name of the NWB file,analysis_file_name name of the file,decoded_position_object_id
231ed383-2f43-fc37-7e28-2e1ebce17873,BraveLu20240505_.nwb,BraveLu20240505_BZ1G49CS6S.nwb,a2619f97-ec05-4e51-a833-2eeed0cdbd3d
fb231218-5693-1d21-6fcd-74d35ea7eefe,IM-1478_20220726_.nwb,IM-1478_20220726_8P96ZLDUTU.nwb,33cb5f8f-a93d-46e6-abff-402605c50e81


decoding_merge_id,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based)
fb231218-5693-1d21-6fcd-74d35ea7eefe,IM-1478_20220726_.nwb,0


decoding_merge_id,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),analysis_file_name name of the file,hex_assignment_object_id
fb231218-5693-1d21-6fcd-74d35ea7eefe,IM-1478_20220726_.nwb,0,IM-1478_20220726_A78DJ305N6.nwb,c5ff0cdd-ea8d-48ac-accd-26debf3d68ee


Grab a `merge_id` that points to the `DecodingOutput` entry you want to use

In [46]:
from spyglass.decoding.decoding_merge import DecodingOutput

decode_key = {"nwb_file_name": nwb_file_name, "decoding_param_name": "contfrag_sorted"}

print("all nwbs in decoded output:")
display(DecodingOutput.merge_get_part(decode_key, multi_source=True))

# By using multi_source=True, this returns a list. So we iterate over them
all_decodes_for_this_nwb = (DecodingOutput.merge_get_part(decode_key, multi_source=True))
# For now just grab the first one
decode_output = all_decodes_for_this_nwb[0]

# Fetch results to make sure they exist
merge_id = decode_output.fetch1("KEY")
results = DecodingOutput.fetch_results(merge_id)

display(results)

print(f"Merge id: {merge_id}")

all nwbs in decoded output:


[*merge_id      nwb_file_name  unit_filter_pa sorted_spikes_ position_group decoding_param encoding_inter decoding_inter estimate_decod
 +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+ +------------+
 fb231218-5693- IM-1478_202207 default_exclus sorted_spikes_ sorted_spikes_ contfrag_sorte 00_r1          epoch0_block1  0             
  (Total: 1)]

[2025-11-10 18:28:13,905][WARNING]: Skipped checksum for file with hash: b5d2356a-75c2-6ae0-7d61-ca822d6905fc, and path: /stelmo/nwb/analysis/IM-1478_20220726/IM-1478_20220726_cde60043-99eb-40f2-bfb6-408f38e54fc9.nc
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/xarray/namedarray/core.py:496: UserWarning: Duplicate dimension names present: dimensions {'states'} appear more than once in dims=('states', 'states'). We do not yet support duplicate dimension names, but we do allow initial construction of the object. We recommend you rename the dims immediately to become distinct, as most xarray functionality is likely to fail silently if you do not. To rename the dimensions you will need to set the ``.dims`` attribute of each variable, ``e.g. var.dims=('x0', 'x1')``.
  warnings.warn(
/home/scrater/miniforge3/envs/spyglass/lib/python3.10/site-packages/xarray/namedarray/core.py:496: UserWarning: Duplicate dimension names present: dimensions {'states'} appear more than onc

<xarray.Dataset> Size: 41GB
Dimensions:                      (time: 807009, state_ind: 12750, dim_0: 12750,
                                  states: 2, intervals: 1, state_bins: 12750)
Coordinates:
  * time                         (time) float64 6MB 49.92 49.93 ... 1.664e+03
  * state_ind                    (state_ind) int32 51kB 0 0 0 0 0 ... 1 1 1 1 1
  * states                       (states) object 16B 'Continuous' 'Fragmented'
    environments                 (states) object 16B ...
    encoding_groups              (states) int32 8B ...
  * state_bins                   (state_bins) object 102kB MultiIndex
  * state                        (state_bins) object 102kB 'Continuous' ... '...
  * x_position                   (state_bins) float64 102kB 12.51 ... 179.5
  * y_position                   (state_bins) float64 102kB 5.012 ... 151.9
Dimensions without coordinates: dim_0, intervals
Data variables:
    initial_conditions           (dim_0) float64 102kB ...
    discrete_state_transitions   (states, states) float64 32B ...
    acausal_posterior            (intervals, time, state_bins) float32 41GB ...
    acausal_state_probabilities  (intervals, time, states) float32 6MB ...
Attributes:
    marginal_log_likelihoods:  -4664679.0

Merge id: {'merge_id': UUID('fb231218-5693-1d21-6fcd-74d35ea7eefe')}


Populate `DecodedPosition`

This gets the most likely decoded x and y position at each time point.
Populating a single entry may take a long time, depending on how long your decoding interval is (30+ mins on breeze)

In [47]:
from hex_maze_decoding import DecodedPosition

# Create a key with the merge_id from DecodingOutput and the nwb_file_name
decoded_pos_key = {
    "decoding_merge_id": str(merge_id["merge_id"]),
    "nwb_file_name": nwb_file_name,
    "epoch" : 0,
}
print(decoded_pos_key)

# Populate DecodedPosition
DecodedPosition.populate(decoded_pos_key)

{'decoding_merge_id': 'fb231218-5693-1d21-6fcd-74d35ea7eefe', 'nwb_file_name': 'IM-1478_20220726_.nwb', 'epoch': 0}


{'success_count': 0, 'error_list': []}

Make sure it worked!

In [48]:
# Show our newly populated entry in the table
display(DecodedPosition() & decoded_pos_key)

# Fetch the df of max likelihood x,y decoded position
decoded_pos_df = (DecodedPosition & decoded_pos_key).fetch1_dataframe()
display(decoded_pos_df)

decoding_merge_id,nwb_file_name name of the NWB file,analysis_file_name name of the file,decoded_position_object_id
fb231218-5693-1d21-6fcd-74d35ea7eefe,IM-1478_20220726_.nwb,IM-1478_20220726_8P96ZLDUTU.nwb,33cb5f8f-a93d-46e6-abff-402605c50e81


,hpd_thresh,spatial_cov,pred_x,pred_y
time,,,,
49.924743,0.000059,1226,123.824107,50.677172
49.926743,0.000059,797,123.824107,50.677172
49.928743,0.000045,225,125.811777,52.662603
49.930743,0.000061,99,125.811777,52.662603
49.932743,0.001249,27,125.811777,52.662603
...,...,...,...,...
1663.932692,0.000166,1694,52.267987,138.036134
1663.934692,0.000154,1622,52.267987,138.036134
1663.936692,0.000158,1728,54.255657,138.036134


### Now assign this decoded position to a hex.

In [49]:
from hex_maze_decoding import DecodedHexPositionSelection, DecodedHexPosition

# Insert into selection table
DecodedHexPositionSelection.insert1(decoded_pos_key, skip_duplicates=True)

# Make sure it worked
display(DecodedHexPositionSelection() & decoded_pos_key)

decoding_merge_id,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based)
fb231218-5693-1d21-6fcd-74d35ea7eefe,IM-1478_20220726_.nwb,0


In [50]:
# Run populate to assign to hex
DecodedHexPosition.populate(decoded_pos_key)

# Make sure it worked
display(DecodedHexPosition() & decoded_pos_key)

decoding_merge_id,nwb_file_name name of the NWB file,epoch the session epoch for this task and apparatus(1 based),analysis_file_name name of the file,hex_assignment_object_id
fb231218-5693-1d21-6fcd-74d35ea7eefe,IM-1478_20220726_.nwb,0,IM-1478_20220726_A78DJ305N6.nwb,c5ff0cdd-ea8d-48ac-accd-26debf3d68ee


In [51]:
# Fetch the df of decode assigned to hex
decoded_hex_df = (DecodedHexPosition & decoded_pos_key).fetch1_dataframe()
display(decoded_hex_df)

,hex,hex_including_sides,distance_from_centroid
time,,,
49.924743,9,9,4.717731
49.926743,9,9,4.717731
49.928743,9,9,2.455686
49.930743,9,9,2.455686
49.932743,9,9,2.455686
...,...,...,...
1663.932692,49,49_left,2.277534
1663.934692,49,49_left,2.277534
1663.936692,49,49_left,3.617987


In [52]:
# Combine both dataframes to get decoded position and also the hex

decode_df  = decoded_pos_df.join(decoded_hex_df)
display(decode_df)

,hpd_thresh,spatial_cov,pred_x,pred_y,hex,hex_including_sides,distance_from_centroid
time,,,,,,,
49.924743,0.000059,1226,123.824107,50.677172,9,9,4.717731
49.926743,0.000059,797,123.824107,50.677172,9,9,4.717731
49.928743,0.000045,225,125.811777,52.662603,9,9,2.455686
49.930743,0.000061,99,125.811777,52.662603,9,9,2.455686
49.932743,0.001249,27,125.811777,52.662603,9,9,2.455686
...,...,...,...,...,...,...,...
1663.932692,0.000166,1694,52.267987,138.036134,49,49_left,2.277534
1663.934692,0.000154,1622,52.267987,138.036134,49,49_left,2.277534
1663.936692,0.000158,1728,54.255657,138.036134,49,49_left,3.617987


## Below this is just other maybe helpful stuff but also you can ignore it.

---------------------------------------------
 Merge keys are hard sometimes. 
 
 I have a helper method `get_all_valid_keys` that finds all valid keys to insert into `DecodedHexPositionSelection`

Valid means the session exists in `HexMazeBlock`, `DecodedPosition`, and `HexCentroids`.

In [53]:
from hex_maze_decoding import DecodedHexPositionSelection, DecodedHexPosition

# Get all valid keys for the DecodedHexPositionSelection table for this nwb
# (valid = the session has HexMazeBlock, DecodedPosition, and HexCentroids data)
all_valid_keys = DecodedHexPositionSelection.get_all_valid_keys(verbose=False)
nwb_file_keys = [key for key in all_valid_keys if key["nwb_file_name"] == nwb_file_name]

if not nwb_file_keys:
    print(f"No valid HexPositionSelection keys found for {nwb_file_name}")

# Insert each key into HexPositionSelection
for key in nwb_file_keys:

    # Skip inserting the key if it already exists in the table
    if key in DecodedHexPositionSelection:
        continue
    try:
        DecodedHexPositionSelection.insert1(key, skip_duplicates=True)
        print(f"Inserted new key {key} into DecodedHexPositionSelection")
    except Exception as e:
        print(f"Skipping insert for {key}: {e}")


In [54]:
# Only populate HexPosition with keys for this nwb
selection_keys = (DecodedHexPositionSelection & {"nwb_file_name": nwb_file_name}).fetch("KEY")
print(selection_keys)
print(f"Populating HexPosition for {len(selection_keys)} entries in {nwb_file_name}")
DecodedHexPosition.populate(selection_keys)

[{'decoding_merge_id': UUID('fb231218-5693-1d21-6fcd-74d35ea7eefe'), 'nwb_file_name': 'IM-1478_20220726_.nwb', 'epoch': 0}]
Populating HexPosition for 1 entries in IM-1478_20220726_.nwb


{'success_count': 0, 'error_list': []}